In [2]:
pip install mesa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model

# Con ''SimultaneousActivation, hacemos que todos los agentes se activen ''al azar''.
from mesa.time import RandomActivation
import random

from mesa.space import ContinuousSpace
from mesa.space import MultiGrid

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricSos.
import numpy as np

In [4]:
class VacuumAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

    def step(self):
        if self.model.room[self.pos] == 1:
            # Aspira la celda
            self.model.room[self.pos] = 0
        else:
            # Mueve a una celda vecina aleatoria
            possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
            new_position = random.choice(possible_steps)
            self.model.grid.move_agent(self, new_position)

In [5]:
class CleaningModel(Model):
    def __init__(self, num_agents, width, height, dirty_percentage):
        super().__init__()
        self.num_agents = num_agents
        self.grid = MultiGrid(width, height, False)
        self.schedule = RandomActivation(self)
        self.current_step = 0
        self.room = {}

        # Inicializa la habitación con celdas sucias
        num_dirty_cells = int(width * height * dirty_percentage)
        dirty_cells = random.sample([(x, y) for x in range(width) for y in range(height)], num_dirty_cells)
        for cell in dirty_cells:
            self.room[cell] = 1
        for cell in [(x, y) for x in range(width) for y in range(height)]:
            if cell not in self.room:
                self.room[cell] = 0

        # Inicializa los agentes
        for i in range(self.num_agents):
            agent = VacuumAgent(i, self)
            self.schedule.add(agent)
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(agent, (x, y))

    def step(self):
        self.schedule.step()
        self.current_step += 1
        if self.calculate_clean_percentage() == 100:
            self.running = False

    def calculate_clean_percentage(self):
        clean_cells = sum(1 for cell in self.room.values() if cell == 0)
        total_cells = self.grid.width * self.grid.height
        clean_percentage = (clean_cells / total_cells) * 100
        return clean_percentage

In [6]:
# Función para crear la animación
def animate_model(model):
    fig, ax = plt.subplots(figsize=(6, 6))

    def update(frame):
        if not model.running:
            return  # Detener la animación si el modelo ha dejado de correr

        ax.clear()
        grid = np.zeros((model.grid.width, model.grid.height))
        for cell in model.room:
            grid[cell] = model.room[cell]
        ax.imshow(grid, cmap='gray')

        ax.set_title(f'Step: {model.current_step}')
        ax.set_xticks([])
        ax.set_yticks([])
        model.step()

    anim = animation.FuncAnimation(fig, update, frames=None, repeat=False)
    plt.close()
    return anim

In [7]:
# Parámetros de la simulación
width = 20
height = 20
dirty_percentage = 0.80
num_agents_list = [1, 5, 10]
check_steps = [500, 1000, 1500]

In [10]:
# Ejecutar la simulación y animar
animations = []  # Lista para mantener las animaciones en memoria
for num_agents in num_agents_list:
    model = CleaningModel(num_agents, width, height, dirty_percentage)

    # Variable para mantener el estado de limpieza de cada paso
    clean_percentage_by_step = {}

    # Ejecutar todos los pasos del modelo
    while model.running:
        model.step()

        # Verificar si el paso actual coincide con los pasos a verificar
        if model.current_step in check_steps:
            clean_percentage = model.calculate_clean_percentage()
            clean_percentage_by_step[model.current_step] = clean_percentage
            print(f"Porcentaje de celdas limpias en el paso {model.current_step} con {num_agents} agentes: {clean_percentage:.2f}%")

    # Mostrar el porcentaje en los pasos especificados aunque se haya alcanzado el 100% antes
    for step in check_steps:
        if step not in clean_percentage_by_step:
            clean_percentage = model.calculate_clean_percentage()
            clean_percentage_by_step[step] = clean_percentage
            print(f"Porcentaje de celdas limpias en el paso {step} con {num_agents} agentes: {clean_percentage:.2f}%")

    # Verificar si se alcanzó el 100% de celdas limpias
    if model.calculate_clean_percentage() == 100:
        print(f"Paso en el que se alcanzó el 100% de celdas limpias con {num_agents} agentes: {model.current_step}")

    anim = animate_model(model)
    animations.append(anim)  # Mantener la animación en memoria

Porcentaje de celdas limpias en el paso 500 con 1 agentes: 48.50%
Porcentaje de celdas limpias en el paso 1000 con 1 agentes: 68.50%
Porcentaje de celdas limpias en el paso 1500 con 1 agentes: 82.50%
Paso en el que se alcanzó el 100% de celdas limpias con 1 agentes: 6208
Porcentaje de celdas limpias en el paso 500 con 5 agentes: 92.75%
Porcentaje de celdas limpias en el paso 1000 con 5 agentes: 100.00%
Porcentaje de celdas limpias en el paso 1500 con 5 agentes: 100.00%
Paso en el que se alcanzó el 100% de celdas limpias con 5 agentes: 959
Porcentaje de celdas limpias en el paso 500 con 10 agentes: 99.75%
Porcentaje de celdas limpias en el paso 1000 con 10 agentes: 100.00%
Porcentaje de celdas limpias en el paso 1500 con 10 agentes: 100.00%
Paso en el que se alcanzó el 100% de celdas limpias con 10 agentes: 535


<ipython-input-6-0cdcdddc0f9b>:20: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  anim = animation.FuncAnimation(fig, update, frames=None, repeat=False)


In [ ]:
anim